In [4]:
!pip install crewai crewai-tools firecrawl-py

In [5]:
from crewai import LLM

llm = LLM(model="ollama/llama3.2",
          base_url="http://localhost:11434")

In [6]:
import yaml

with open('config/planner_agents.yaml','r') as f:
    agents_config = yaml.safe_load(f)

with open('config/planner_tasks.yaml','r') as f:
    tasks_config = yaml.safe_load(f)

In [7]:
from pydantic import BaseModel
from typing import Optional

class Tweet(BaseModel):
    """Reprsents and individual tweet in a thread"""
    content: str
    is_hook: bool = False
    media_urls: Optional[list[str]] = []

class Thread(BaseModel):
    """Represents a Twitter thread"""
    topic: str
    tweets: list[Tweet] = []

In [8]:
from pydantic import BaseModel
from typing import Optional

class LinkedInPost(BaseModel):
    """Represents a LinkedIn post"""
    content: str
    media_urls: str

In [9]:
from crewai_tools import DirectoryReadTool, FileReadTool

all_tools = [DirectoryReadTool, FileReadTool]


In [10]:
from crewai import Agent, Task

draft_analyzer = Agent(config=agents_config['draft_analyzer'],tools=all_tools,llm=llm)
analyze_draft_task = Task(config=tasks_config['analyze_draft'],agent=draft_analyzer)

twitter_thread_planner = Agent(config=agents_config['twitter_thread_planner'],tools=all_tools,llm=llm)
create_twitter_thread_plan = Task(config=tasks_config['create_twitter_thread_plan'],agent=twitter_thread_planner,output_pydantic=Thread)


linked_post_planner = Agent(config=agents_config['linkedin_post_planner'],tools=all_tools,llm=llm)
create_linkedin_post_plan = Task(config=tasks_config['create_linkedin_post_plan'],agent=linked_post_planner,output_pydantic=LinkedInPost)


ValidationError: 2 validation errors for Agent
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<class 'crewai_tools.tool...tool.DirectoryReadTool'>, input_type=ModelMetaclass]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
tools.1
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value=<class 'crewai_tools.tool...read_tool.FileReadTool'>, input_type=ModelMetaclass]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [ ]:
from pydantic import BaseModel
from pathlib import Path

class ContentPlanningState(BaseModel):
    """
    State for the content planning flow
    """
    # URL of the blog to scrape
    blog_post_url: str = blog_post_url
    
    # Path where the scraped content will be stored
    draft_path: Path = "assets/"
    
    # Determines whether to create a Twitter or LinkedIn post 
    post_type: str = "twitter"  
    
    # Example Twitter threads for style reference
    path_to_example_threads: str = "assets/example_threads.txt" 
    
    # Example LinkedIn posts for reference
    path_to_example_linkedin: str = "assets/example_linkedin.txt"

In [ ]:
from firecrawl import FirecrawlApp
import os
import uuid

class CreateContentPlanningFlow(Flow[ContentPlanningState]):

    @start()
    def scrape_blog_post(self):
        print(f"# Fetching draft from: {self.state.blog_post_url}")

        # Initialize FireCrawl
        app = FirecrawlApp(api_key=os.getenv("FIRECRAWL_API_KEY"))
        
        # Scrape the blog post in Markdown and HTML format
        scrape_result = app.scrape_url(self.state.blog_post_url,
                                       params={'formats': ['markdown', 'html']})

        # Extract the title (fallback to a UUID if not found)
        try:
            title = scrape_result['metadata']['title']
        except Exception:
            title = str(uuid.uuid4())

        # Store the scraped content as a markdown file
        self.state.draft_path = f'assets/{title}.md'
        with open(self.state.draft_path, 'w') as f:
            f.write(scrape_result['markdown'])

        return self.state
    
    @router(scrape_blog_post)
    def select_platform(self):
        if self.state.post_type == "twitter":
            return "twitter"
        elif self.state.post_type == "linkedin":
            return "linkedin"
        
    @listen("twitter")
    def twitter_draft(self):
        print(f"planning Twitter post for : {self.state.draft_path}")
        result = twitter_planning_crew,kickoff(inputs={
            "draft_path": str(self.state.draft_path),
            "path_to_example_threads": self.state.path_to_example_threads
        })
        print(f"#Planned Twitter post for {self.state.draft_path}")

        for i,tweet in enumerate(result.pydantic.tweets):
            print(f"Tweet {i+1}: {tweet.content}\n")
            print(f"Media Urls: {tweet.media_urls}\n")
            print("-"*100)

        return result
    
    @listen("linkedin")
    def linkedin_draft(self):
        print(f"# Planning content for: {self.state.draft_path}")
    
        # Execute the LinkedIn Planning Crew
        result = linkedin_planning_crew.kickoff(inputs={
            'draft_path': self.state.draft_path, 
            'path_to_example_linkedin': self.state.path_to_example_linkedin
        })
    
        print(f"# Planned content for {self.state.draft_path}:")
        print(f"{result.pydantic.content}")
    
        return result

    @listen(or_(twitter_draft, linkedin_draft))
    def save_plan(self,plan):
        with open(f"output/draft.json", 'w') as f:
            json.dump(plan.pydantic.model_dump(), f, indent=2)



flow = CreateContentPlanningFlow()
flow.plot()
flow.kickoff()
blog_post_url = "https://blog.dailydoseofds.com/p/5-chunking-strategies-for-rag"
draft_path = "assets/"
post_type = "twitter"
path_to_example_threads = "assets/example_threads.txt"
path_to_example_linkedin = "assets/example_linkedin.txt"
flow.state.post_type = "linkedin"
flow.kickoff()


In [13]:
cd /book_flow

    

NameError: name 'cd' is not defined